In [1]:
import mysql.connector

# Connect to MySQL running locally
conn = mysql.connector.connect(
    host="localhost",  # 'localhost' since it's on your PC
    user="root",  # default XAMPP username is 'root'
    password="",  # default XAMPP has no password
    database="macromed",  # replace with your actual DB name
    port=3306,  # default port
)

# Check connection
if conn.is_connected():
    print("✅ Connected to the local MySQL database")

# Example query
cursor = conn.cursor()
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()

print("📋 Tables in the database:")
for table in tables:
    print(table)

# Cleanup
cursor.close()
conn.close()

✅ Connected to the local MySQL database
📋 Tables in the database:
('cache',)
('cache_locks',)
('interactions',)
('migrations',)
('personal_access_tokens',)
('products',)
('reviews',)
('sessions',)
('users',)


In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Replace with your credentials
host = "localhost"
user = "root"
password = ""  # leave blank if no password
database = "macromed"

# Create engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# Load products table
products_df = pd.read_sql("SELECT * FROM products", engine)

# Preview
print(products_df.head())

   product_id                                       product_name  \
0           1  8 Pcs Hegar Uterine Sounds Dilator Set 3mm-18m...   
1           2  Metzenbaum Scissors 5.75″ Curved Surgical Vete...   
2           3  Adson Tissue Forceps 1×2 Rat Tooth 4.75″ Fine ...   
3           4  Tactical Black Lister Bandage Scissors 5.5″ | ...   
4           5  Medical Skin Suture Practice Silicone Pad | SM...   

                                         description              category  \
0  Voluptates et voluptas consequuntur quod.\nThi...            Gynecology   
1  Dolorem officia praesentium explicabo laborios...  Surgical Instruments   
2  Ut voluptate aliquam omnis facilis repellat ni...    Dissecting Forceps   
3  Fuga quo doloribus sit omnis aut placeat recus...  Surgical Instruments   
4  Impedit aut omnis voluptate commodi est itaque...               General   

                 subcategory  price         brand               manufacturer  \
0  Neurosurgical Instruments  39.99     Su

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import HTML
import pandas as pd


class Recommendations:
    def __init__(self, products_df):
        products_df.fillna("", inplace=True)
        products_df["combined_text"] = (
            products_df["product_name"]
            + " "
            + products_df["description"]
            + " "
            + products_df["category"]
            + " "
            + products_df["subcategory"]
            + " "
            + products_df["brand"]
            + " "
            + products_df["material"]
        )

        self.products_df = products_df
        self.product_indices = pd.Series(
            products_df.index, index=products_df["product_id"]
        )
        self.tfidf = TfidfVectorizer(stop_words="english")
        self.tfidf_matrix = self.tfidf.fit_transform(products_df["combined_text"])
        self.cosine_sim = cosine_similarity(self.tfidf_matrix, self.tfidf_matrix)

    def get_product_to_product_recommendations(self, product_id):
        if product_id not in self.product_indices:
            return f"Product ID {product_id} not found."

        idx = self.product_indices[product_id]
        target = self.products_df.iloc[idx]

        sim_scores = list(enumerate(self.cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:]  # remove self

        recommendations = []

        def filter_and_add(condition, count):
            nonlocal recommendations, sim_scores
            added = 0
            for i, _ in sim_scores:
                candidate = self.products_df.iloc[i]
                if (
                    condition(candidate)
                    and candidate["product_id"] != product_id
                    and candidate["product_id"]
                    not in [r["product_id"] for r in recommendations]
                ):
                    recommendations.append(candidate)
                    added += 1
                    if added == count:
                        break

        # 2 from same category or subcategory
        filter_and_add(
            lambda p: (
                p["category"] == target["category"]
                or p["subcategory"] == target["subcategory"]
            ),
            2,
        )

        # 2 from same brand
        filter_and_add(lambda p: p["brand"] == target["brand"], 2)

        # 1 from same material
        filter_and_add(lambda p: p["material"] == target["material"], 1)

        return pd.DataFrame(recommendations)[
            [
                "product_id",
                "product_name",
                "category",
                "brand",
                "material",
                "price",
                "image_url",
                "product_url",
            ]
        ]

    def get_price_based_recommendations(self, product_id, top_n=5):
        if product_id not in self.product_indices:
            return f"Product ID {product_id} not found."

        target_price = self.products_df.loc[
            self.products_df["product_id"] == product_id, "price"
        ].values[0]

        price_df = self.products_df.copy()
        price_df["price_diff"] = (price_df["price"] - target_price).abs()
        price_df = price_df[price_df["product_id"] != product_id]
        closest = price_df.sort_values(by="price_diff").head(top_n)

        return closest[
            [
                "product_id",
                "product_name",
                "category",
                "brand",
                "material",
                "price",
                "image_url",
                "product_url",
            ]
        ]

    def display_products_with_images(self, df, max_items=5):
        df = df.copy().head(max_items)
        html = "<table><tr>"

        for _, row in df.iterrows():
            html += f"""
            <td style="text-align:center; padding:10px">
                <a href="{row['product_url']}" target="_blank">
                    <img src="{row['image_url']}" width="120"><br>
                    <b>{row['product_name']}</b>
                </a><br>
                <i>{row['category']}</i><br>
                <span style="color:green; font-weight:bold;">${row['price']:.2f}</span><br>
            </td>
            """
        html += "</tr></table>"
        return HTML(html)


rec = Recommendations(products_df)

# Get and display content-based recommendations
product_recs = rec.get_product_to_product_recommendations(product_id=1)
rec.display_products_with_images(product_recs)

# Get and display price-based recommendations
price_recs = rec.get_price_based_recommendations(product_id=1)
rec.display_products_with_images(price_recs)

,product_id,product_name,category,brand,material,price,image_url,product_url
2062,2063,Hegar Uterine Dilator 17mm-18mm Double Ended 7...,Gynecology,Meditek,Medical Grade Stainless Steel,16.35,Missing_URL,https://surgicalmart.com/shop/surgical-instrum...
998,999,Hegar Uterine Dilator 3mm-4mm Double Ended 7.5...,Gynecology,SurgiCare,Tungsten Carbide,16.35,Missing_URL,https://surgicalmart.com/shop/surgical-instrum...
779,780,Hegar Uterine Dilator 5mm-6mm Double Ended 7.5...,Gynecology,SurgiCare,Medical Grade Stainless Steel,16.35,https://surgicalmart.com/wp-content/uploads/20...,https://surgicalmart.com/shop/surgical-instrum...
71,72,8 Pcs Pratt Uterine Dilator Set Curved 11.5″ D...,Gynecology,SurgiCare,Aluminum Alloy,45.44,Missing_URL,https://surgicalmart.com/shop/surgical-instrum...
122,123,14 Pcs Hegar Uterine Dilator Sounds Set 7.5″ S...,Gynecology,BioMatrix,High-Grade Polymer,54.52,https://surgicalmart.com/wp-content/uploads/20...,https://surgicalmart.com/shop/surgical-instrum...


8 Pcs Hegar Uterine Dilator Set 7.5″ Single Ended | SM2468 Gynecology $39.99,Vaginal Lateral Retractor 10.5″ Gold Handle Closed Elongated Shanks | SM2569 Gynecology $39.99,NABATOFF Vein Stripper Set with Sterilizer Case | SM4592 Cardiovascular And Thoracic Instruments $39.99,Plate and Bone Holding Forceps 5.5″ with Footplate & Gold Handle | SM5164 Bone Forceps $39.99,Semm Vaginal Speculum Large 100mmx25mm | SM2648 Gynecology $39.99
